<a href="https://www.kaggle.com/code/shreyanshmanavshukla/predict-calorie-expenditure-using-rf-and-xgboost?scriptVersionId=252481019" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


# Data Import

In [2]:
Train = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
Test = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [3]:
Train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [4]:
Test.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [5]:
print(Train.shape)
print(Test.shape)

(750000, 9)
(250000, 8)


# Feature Extraction

In [6]:
df = Train
X = df.drop('Calories', axis=1)  # Features (everything except 'Calories')
y = df['Calories']               # Target variable


In [7]:
X

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,0,male,36,189.0,82.0,26.0,101.0,41.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7
2,2,female,51,161.0,64.0,7.0,84.0,39.8
3,3,male,20,192.0,90.0,25.0,105.0,40.7
4,4,female,38,166.0,61.0,25.0,102.0,40.6
...,...,...,...,...,...,...,...,...
749995,749995,male,28,193.0,97.0,30.0,114.0,40.9
749996,749996,female,64,165.0,63.0,18.0,92.0,40.5
749997,749997,male,60,162.0,67.0,29.0,113.0,40.9
749998,749998,male,45,182.0,91.0,17.0,102.0,40.3


# Preprocess

In [8]:
# Assuming df is your DataFrame
X = pd.get_dummies(X, columns=['Sex'], dtype=int)

X = X.drop('Sex_female',axis=1)

# Rename 'Sex_male' to 'Sex'
X.rename(columns={'Sex_male': 'Sex'}, inplace=True)
# Sex male:1 female:0

In [9]:
from sklearn.preprocessing import MinMaxScaler


# Separate 'id' column
id_col = X['id']

# Select features to normalize (excluding 'id')
features = X.drop(columns=['id'])

# Apply MinMaxScaler
scaler = MinMaxScaler()
normalized_values = scaler.fit_transform(features)

# Create a new DataFrame with normalized values
normalized_df = pd.DataFrame(normalized_values, columns=features.columns)

# Add back the 'id' column
X = pd.concat([id_col, normalized_df], axis=1)

In [10]:
X

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex
0,0,0.271186,0.656250,0.479167,0.862069,0.557377,0.886364,1.0
1,1,0.745763,0.385417,0.250000,0.241379,0.295082,0.590909,0.0
2,2,0.525424,0.364583,0.291667,0.206897,0.278689,0.613636,0.0
3,3,0.000000,0.687500,0.562500,0.827586,0.622951,0.818182,1.0
4,4,0.305085,0.416667,0.260417,0.827586,0.573770,0.795455,0.0
...,...,...,...,...,...,...,...,...
749995,749995,0.135593,0.697917,0.635417,1.000000,0.770492,0.863636,1.0
749996,749996,0.745763,0.406250,0.281250,0.586207,0.409836,0.772727,0.0
749997,749997,0.677966,0.375000,0.322917,0.965517,0.754098,0.863636,1.0
749998,749998,0.423729,0.583333,0.572917,0.551724,0.573770,0.727273,1.0


# Data Spliting

In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Training

## Random Forest

In [12]:
RF_model = RandomForestRegressor(n_estimators=200)

In [13]:
# Train the model
RF_model.fit(X_train.iloc[:,1::], y_train)

RandomForestRegressor(n_estimators=200)

In [14]:
import joblib

# Save the trained model to a file
# joblib.dump(RF_model, 'random_forest_model.pkl')
# Load the model from the file
# RF_model = joblib.load('random_forest_model.pkl')


In [15]:
y_pred = RF_model.predict(X_test.iloc[:,1:])

# 9. Evaluate the model
y_pred = np.maximum(0, y_pred)  # Prevent negative predictions for regression task

rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {rmsle:.4f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")


RMSLE: 0.0632
MSE: 14.52
R² Score: 0.9963


## XGBoost

In [16]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',  # Regression task
    booster='gbtree',  # Tree booster
    n_estimators=500,  # Number of trees
    learning_rate=0.1,  # Learning rate
    max_depth=8,  # Max depth of trees
    subsample=0.8,  # Subsample ratio of the training set
    colsample_bytree=0.8  # Subsample ratio of columns when building each tree
)

# 6. Train the model
xgb_model.fit(X_train.iloc[:,1::], y_train)


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [17]:
import joblib

# Save the trained model to a file
# joblib.dump(xgb_model, 'xgb_model.pkl')
# Load the model from the file
# xgb_model = joblib.load('xgb_model.pkl')


In [18]:
y_pred = xgb_model.predict(X_test.iloc[:,1::])

# 9. Evaluate the model
y_pred = np.maximum(0, y_pred)  # Prevent negative predictions for regression task

rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {rmsle:.4f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")


RMSLE: 0.0612
MSE: 12.92
R² Score: 0.9967


# Ensemble 

In [19]:
xgb_pred = xgb_model.predict(X_test.iloc[:,1::])
rf_pred = RF_model.predict(X_test.iloc[:,1::])

# Simple average ensemble
ensemble_pred = (xgb_pred + rf_pred) / 2

## Result

In [20]:


rmsle = np.sqrt(mean_squared_log_error(y_test, ensemble_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSLE: {rmsle:.4f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.4f}")


RMSLE: 0.0605
MSE: 12.92
R² Score: 0.9967


# Submission File

In [21]:
Test

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5
...,...,...,...,...,...,...,...,...
249995,999995,female,56,159.0,62.0,6.0,85.0,39.4
249996,999996,male,32,202.0,101.0,3.0,84.0,38.4
249997,999997,female,31,164.0,64.0,14.0,98.0,40.1
249998,999998,female,62,158.0,61.0,25.0,106.0,40.7


In [22]:
# Assuming df is your DataFrame
X = pd.get_dummies(Test, columns=['Sex'], dtype=int)

X = X.drop('Sex_female',axis=1)

# Rename 'Sex_male' to 'Sex'
X.rename(columns={'Sex_male': 'Sex'}, inplace=True)
# Sex male:1 female:0

In [23]:
# Separate 'id' column
id_col = X['id']

# Select features to normalize (excluding 'id')
features = X.drop(columns=['id'])

# Apply MinMaxScaler
scaler = MinMaxScaler()
normalized_values = scaler.fit_transform(features)

# Create a new DataFrame with normalized values
normalized_df = pd.DataFrame(normalized_values, columns=features.columns)

# Add back the 'id' column
X = pd.concat([id_col, normalized_df], axis=1)

In [24]:
xgb_pred = xgb_model.predict(X.iloc[:,1::])
rf_pred = RF_model.predict(X.iloc[:,1::])

# Simple average ensemble
ensemble_pred = (xgb_pred + rf_pred) / 2

In [25]:
submission = pd.DataFrame({
    'id': X['id'].values,  # Ensure 'id' was retained in X_test
    'Calories': ensemble_pred
})
submission.to_csv("submission.csv", index=False)

submission

,id,Calories
0,750000,26.924606
1,750001,109.415404
2,750002,86.770824
3,750003,126.822753
4,750004,76.022042
...,...,...
249995,999995,26.341252
249996,999996,9.373592
249997,999997,73.293406
249998,999998,169.789512
